In [10]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "google/flan-t5-large"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at google/flan-t5-large and are newly initialized: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
prompt_1 = "What is the capital of France?"
prompt_2 = "Cuál es la capital de Colombia?"

inputs_1 = tokenizer(prompt_1, return_tensors="pt")
inputs_1

{'input_ids': tensor([[ 363,   19,    8, 1784,   13, 1410,   58,    1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}

In [3]:
inputs_2 = tokenizer(prompt_2, return_tensors="pt")
inputs_2

{'input_ids': tensor([[ 1839,  2975,    40,     3,    15,     7,    50,  1784,    20, 18898,
            58,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
outputs_1 = model(**inputs_1)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [8]:
inputs_1 = {k: v.to(device) for k, v in inputs_1.items()}
inputs_2 = {k: v.to(device) for k, v in inputs_2.items()}
inputs_1

{'input_ids': tensor([[ 363,   19,    8, 1784,   13, 1410,   58,    1]], device='cuda:0'),
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [14]:
completion_raw = model.generate(**inputs_1, max_new_tokens=100)
completion_raw

tensor([[  0, 260, 159,   1]], device='cuda:0')

In [16]:
tokenizer.decode(completion_raw[0], skip_special_tokens=True)

'paris'

In [17]:
completion_raw_2 = model.generate(**inputs_2, max_new_tokens=100)
tokenizer.decode(completion_raw_2[0], skip_special_tokens=True)

'Bogotá'